In [1]:
from __future__ import division, print_function, absolute_import

import collections
import os
import random
import urllib.request
import zipfile

import numpy as np
import tensorflow as tf

In [2]:
import urllib.request
import os
import tarfile

url = "http://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz"
filepath = "aclImdb_v1.tar.gz"
if not os.path.isfile(filepath):
    result = urllib.request.urlretrieve(url, filepath)
    print('downloaded: ', result)

# 解壓縮檔案
# Open for reading with gzip compression
if not os.path.exists("data/aclImdb"):
    tfile = tarfile.open("aclImdb_v1.tar.gz", 'r:gz')
    result = tfile.extractall('data/')

#from keras.preprocessing import sequence
#from keras.preprocessing.text import Tokenizer

# Regular expression operations
# Compile a regular expression pattern into a regular expression object
import re
def rm_tags(text):
    re_tag = re.compile(r'<[^>]+>')
    return re_tag.sub('', text)
import os
def read_files(filetype):
    path = "data/aclImdb/"
    file_list = []
    
    positive_path = path + filetype + "/pos/"
    for f in os.listdir(positive_path):
        file_list += [positive_path + f]
    
    negative_path = path + filetype + "/neg/"
    for f in os.listdir(negative_path):
        file_list += [negative_path + f]
    
    print('read ', filetype, ' files: ', len(file_list))
    
    all_labels = ([1] * 12500 + [0] * 12500)
    
    all_texts = []
    for fi in file_list:
        with open(fi, encoding = 'utf8') as file_input:
            all_texts += [rm_tags(" ".join(file_input.readlines()))]
    
    return all_labels, all_texts

y_train, train_text = read_files("train")
y_test, test_text = read_files("test")
print(train_text[0])
print(y_train[0])
print(train_text[12501])
print(y_train[12501])
print(test_text[0])
print(y_test[0])
print(test_text[12501])
print(y_test[12501])


read  train  files:  25000
read  test  files:  25000
An intriguingly bold film weaves the seemingly effortless camerawork with some superb casting and an explosive soundtrack to plot the damaging effects of the crime and corruption of the Santiago underworld on 2 naive young brothers from the southern city of Temuco.Film debutant Daniella Rios is the seductive erotic dancer Gracia, working in the nightclub owned by the face of the new mini-wave in Chilean film production, Alejandro Trejo. The elder brother, played maturely by Nestor Cantillana, is easily convinced to become Trejo's lead henchman, after a night at the stripclub to celebrate younger brother Victor's (Juan Pablo Miranda) seventeenth birthday. From the establishing shot of this opening scene, the film explodes into neo-noir exploration of everything the outside world doesn't usually expect to see in this country so stereotypically conservative and catholic.Gracia's charms of seduction attract the three men like bees to hon

In [3]:
a=""
for i in range(len(train_text)):
    for j in range(len(train_text[i])):
        _tmp=str(train_text[i][j])
        if(_tmp==" "):
            a+=" "
        elif(str(train_text[i][j]).isalpha()==True):
            a+=train_text[i][j]
a=a.lower()
a=a.split( )
try:
    a=a.remove([])
except:
    pass

In [4]:
import tensorflow.compat.v1 as tf
tf.disable_v2_behavior()



# Training Parameters
learning_rate = 0.1
batch_size = 128
num_steps = 300000
display_step = 10000
eval_step = 200000

# Evaluation Parameters
eval_words = ['atrociously', 'of', 'country', 'judge', 'sight', 'notions']

# Word2Vec Parameters
embedding_size = 200 # Dimension of the embedding vector
max_vocabulary_size = 50000 # Total number of different words in the vocabulary
min_occurrence = 10 # Remove all words that does not appears at least n times
skip_window = 3 # How many words to consider left and right
num_skips = 2 # How many times to reuse an input to generate a label
num_sampled = 64 # Number of negative examples to sample


# Download a small chunk of Wikipedia articles collection
url = 'http://mattmahoney.net/dc/text8.zip'
data_path = 'text8.zip'
if not os.path.exists(data_path):
    print("Downloading the dataset... (It may take some time)")
    filename, _ = urllib.request.urlretrieve(url, data_path)
    print("Done!")
# Unzip the dataset file. Text has already been processed
#data_path = 'aclImdb_v1.tar.gz'

#with zipfile.ZipFile(data_path) as f:
#    text_words = f.read(f.namelist()[0]).lower().split()

text_words=a

# Build the dictionary and replace rare words with UNK token
count = [('UNK', -1)]
# Retrieve the most common words
count.extend(collections.Counter(text_words).most_common(max_vocabulary_size - 1))
# Remove samples with less than 'min_occurrence' occurrences
for i in range(len(count) - 1, -1, -1):
    if count[i][1] < min_occurrence:
        count.pop(i)
    else:
        # The collection is ordered, so stop when 'min_occurrence' is reached
        break
# Compute the vocabulary size
vocabulary_size = len(count)
# Assign an id to each word
word2id = dict()
for i, (word, _)in enumerate(count):
    #print(word)
    #print(i)
    word2id[word] = i

data = list()
unk_count = 0
for word in text_words:
    # Retrieve a word id, or assign it index 0 ('UNK') if not in dictionary
    index = word2id.get(word, 0)
    #print(word)
    #print(index)
    if index == 0:
        unk_count += 1
    data.append(index)
count[0] = ('UNK', unk_count)
id2word = dict(zip(word2id.values(), word2id.keys()))

print("Words count:", len(text_words))
print("Unique words:", len(set(text_words)))
print("Vocabulary size:", vocabulary_size)
print("Most common words:", count[:10])

data_index = 0
# Generate training batch for the skip-gram model
def next_batch(batch_size, num_skips, skip_window):
    global data_index
    assert batch_size % num_skips == 0
    assert num_skips <= 2 * skip_window
    batch = np.ndarray(shape=(batch_size), dtype=np.int32)
    labels = np.ndarray(shape=(batch_size, 1), dtype=np.int32)
    # get window size (words left and right + current one)
    span = 2 * skip_window + 1
    buffer = collections.deque(maxlen=span)
    if data_index + span > len(data):
        data_index = 0
    buffer.extend(data[data_index:data_index + span])
    data_index += span
    for i in range(batch_size // num_skips):
        context_words = [w for w in range(span) if w != skip_window]
        words_to_use = random.sample(context_words, num_skips)
        for j, context_word in enumerate(words_to_use):
            batch[i * num_skips + j] = buffer[skip_window]
            labels[i * num_skips + j, 0] = buffer[context_word]
        if data_index == len(data):
            buffer.extend(data[0:span])
            data_index = span
        else:
            buffer.append(data[data_index])
            data_index += 1
    # Backtrack a little bit to avoid skipping words in the end of a batch
    data_index = (data_index + len(data) - span) % len(data)
    return batch, labels


# Input data
X = tf.placeholder(tf.int32, shape=[None])
# Input label
Y = tf.placeholder(tf.int32, shape=[None, 1])

# Ensure the following ops & var are assigned on CPU
# (some ops are not compatible on GPU)
with tf.device('/cpu:0'):
    # Create the embedding variable (each row represent a word embedding vector)
    embedding = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    # Lookup the corresponding embedding vectors for each sample in X
    X_embed = tf.nn.embedding_lookup(embedding, X)

    # Construct the variables for the NCE loss
    nce_weights = tf.Variable(tf.random_normal([vocabulary_size, embedding_size]))
    nce_biases = tf.Variable(tf.zeros([vocabulary_size]))

# Compute the average NCE loss for the batch
loss_op = tf.reduce_mean(
    tf.nn.nce_loss(weights=nce_weights,
                   biases=nce_biases,
                   labels=Y,
                   inputs=X_embed,
                   num_sampled=num_sampled,
                   num_classes=vocabulary_size))

# Define the optimizer
optimizer = tf.train.GradientDescentOptimizer(learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluation
# Compute the cosine similarity between input data embedding and every embedding vectors
X_embed_norm = X_embed / tf.sqrt(tf.reduce_sum(tf.square(X_embed)))
embedding_norm = embedding / tf.sqrt(tf.reduce_sum(tf.square(embedding), 1, keepdims=True))
cosine_sim_op = tf.matmul(X_embed_norm, embedding_norm, transpose_b=True)

# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

with tf.Session() as sess:

    # Run the initializer
    sess.run(init)

    # Testing data
    for w in eval_words:
        print(w)
        bytesW = w
        #i = word2id.get(bytesW, 0)
        i = word2id[w]
        print(i)

    x_test = np.array([word2id[w] for w in eval_words])

    average_loss = 0
    for step in range(1, num_steps + 1):
        # Get a new batch of data
        batch_x, batch_y = next_batch(batch_size, num_skips, skip_window)
        # Run training op
        _, loss = sess.run([train_op, loss_op], feed_dict={X: batch_x, Y: batch_y})
        average_loss += loss

        if step % display_step == 0 or step == 1:
            if step > 1:
                average_loss /= display_step
            print("Step " + str(step) + ", Average Loss= " + \
                  "{:.4f}".format(average_loss))
            average_loss = 0

        # Evaluation
        if step % eval_step == 0 or step == 1:
            print("Evaluation...")
            sim = sess.run(cosine_sim_op, feed_dict={X: x_test})
            for i in range(len(eval_words)):
                top_k = 8  # number of nearest neighbors
                nearest = (-sim[i, :]).argsort()[1:top_k + 1]
                log_str = '"%s" nearest neighbors:' % eval_words[i]
                for k in range(top_k):
                    log_str = '%s %s,' % (log_str, id2word[nearest[k]])
                print(log_str)

Instructions for updating:
non-resource variables are not supported in the long term
Words count: 5670799
Unique words: 150779
Vocabulary size: 20421
Most common words: [('UNK', 233655), ('the', 326569), ('and', 161507), ('a', 160637), ('of', 145116), ('to', 134715), ('is', 106702), ('in', 91840), ('it', 75136), ('this', 69655)]
atrociously
19680
of
4
country
697
judge
1841
sight
1678
notions
8761
Step 1, Average Loss= 487.0330
Evaluation...
"atrociously" nearest neighbors: silberling, rideau, piano, idyllic, introduces, approve, executives, submarine,
"of" nearest neighbors: cains, nostril, traditions, pain, shrinking, sheila, thief, casper,
"country" nearest neighbors: candle, mechanic, willard, wyatt, humiliated, transmitted, bust, reputations,
"judge" nearest neighbors: foxy, grieco, deux, scarlet, vincenzo, archaeologist, drinking, cruz,
"sight" nearest neighbors: squirrel, complication, hicks, instead, advises, recognise, slams, readings,
"notions" nearest neighbors: hanged, meri